# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np

In [2]:
import os 

# DROP TABLES

songplay_table_drop = "DROP TABLE IF EXISTS songplays;"
user_table_drop = "DROP TABLE IF EXISTS users;"
song_table_drop = "DROP TABLE IF EXISTS songs;"
artist_table_drop = "DROP TABLE IF EXISTS artists;"
time_table_drop = "DROP TABLE IF EXISTS time;"

# CREATE TABLES

songplay_table_create = "CREATE TABLE IF NOT EXISTS songplays ( songplay_id SERIAL PRIMARY KEY, \
                                                                start_time TIMESTAMP NOT NULL REFERENCES time(start_time), \
                                                                user_id TEXT NOT NULL REFERENCES users(user_id), \
                                                                level TEXT NOT NULL, \
                                                                song_id TEXT REFERENCES songs(song_id), \
                                                                artist_id TEXT REFERENCES artists(artist_id), \
                                                                session_id INT NOT NULL, \
                                                                location VARCHAR, \
                                                                user_agent VARCHAR );"

user_table_create = "CREATE TABLE IF NOT EXISTS users ( user_id VARCHAR PRIMARY KEY, \
                                                        first_name VARCHAR NOT NULL, \
                                                        last_name VARCHAR NOT NULL, \
                                                        gender CHAR, \
                                                        level VARCHAR NOT NULL );"

song_table_create = "CREATE TABLE IF NOT EXISTS songs ( song_id VARCHAR PRIMARY KEY, \
                                                        title VARCHAR NOT NULL, \
                                                        artist_id VARCHAR NOT NULL, \
                                                        year INT, \
                                                        duration FLOAT );"

artist_table_create = "CREATE TABLE IF NOT EXISTS artists ( artist_id VARCHAR PRIMARY KEY, \
                                                            name VARCHAR NOT NULL, \
                                                            location VARCHAR, \
                                                            latitude FLOAT, \
                                                            longitude FLOAT );"

time_table_create = "CREATE TABLE IF NOT EXISTS time (  start_time TIMESTAMP PRIMARY KEY, \
                                                        hour INT NOT NULL, \
                                                        day INT NOT NULL, \
                                                        week INT NOT NULL, \
                                                        month INT NOT NULL, \
                                                        year INT NOT NULL, \
                                                        weekday INT NOT NULL );"

# INSERT RECORDS

songplay_table_insert = " INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) \
                          VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"

user_table_insert = " INSERT INTO users (user_id, first_name, last_name, gender, level) \
    VALUES(%s, %s, %s, %s, %s) \
    ON CONFLICT (user_id) \
    DO UPDATE \
    SET level = EXCLUDED.level;"

song_table_insert = " INSERT INTO songs (song_id, title, artist_id, year, duration) \
    VALUES(%s, %s, %s, %s, %s) \
    ON CONFLICT (song_id) \
    DO NOTHING;"

artist_table_insert = " INSERT INTO artists (artist_id, name, location, latitude, longitude) \
    VALUES(%s, %s, %s, %s, %s) \
    ON CONFLICT (artist_id) \
    DO NOTHING;"

time_table_insert = " INSERT INTO time (start_time, hour, day, week, month, year, weekday) \
    VALUES(%s, %s, %s, %s, %s, %s, %s) \
    ON CONFLICT (start_time) \
    DO NOTHING;"


# FIND SONGS

song_select = " SELECT s.song_id, a.artist_id  \
        FROM songs as s  \
        JOIN artists as a  \
            ON s.artist_id = a.artist_id \
        WHERE s.title = %s  \
            AND a.name = %s  \
            AND s.duration = %s; "

# QUERY LISTS

create_table_queries = [user_table_create, song_table_create, artist_table_create, time_table_create, songplay_table_create]
drop_table_queries = [songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]

In [3]:
conn = psycopg2.connect("host=127.0.0.1 port=15432 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('../../data/song_data')

In [6]:
filepath = song_files[0]
print(filepath)

/Users/sergiodacosta/Documents/_github/udacity-de/postgres-data-modeling/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = tuple(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
song_data

('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = tuple(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files('../../data/log_data')

In [13]:
filepath = log_files[0]
print(filepath)

/Users/sergiodacosta/Documents/_github/udacity-de/postgres-data-modeling/data/log_data/2018/11/2018-11-11-events.json


In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df[df['page'] == 'NextSong'].reset_index(drop=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [16]:
t = pd.to_datetime(df.ts, unit="ms")
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
3   2018-11-11 04:36:13.796
4   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [17]:
ts = df.ts
ts.head()

0    1541903636796
1    1541903770796
2    1541904034796
3    1541910973796
4    1541911006796
Name: ts, dtype: int64

In [18]:
start_time = t
hour = t.dt.hour
day = t.dt.day
week = t.dt.isocalendar().week
month = t.dt.month
year = t.dt.year
weekday = t.dt.weekday

In [19]:
time_data = [start_time, hour, day, week, month, year, weekday]
column_labels = ['start_time','hour','day','weekofyear','month','year','weekday']

In [20]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data))).reset_index(drop=True)
time_df.head()

,start_time,hour,day,weekofyear,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796,2,11,45,11,2018,6
2,2018-11-11 02:40:34.796,2,11,45,11,2018,6
3,2018-11-11 04:36:13.796,4,11,45,11,2018,6
4,2018-11-11 04:36:46.796,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates() # remove duplicates
user_df

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,75,Joseph,Gutierrez,M,free
5,92,Ryann,Smith,F,free
10,49,Chloe,Cuevas,F,free
18,88,Mohammad,Rodriguez,M,free
20,96,Cierra,Finley,F,free
21,26,Ryan,Smith,M,free
24,44,Aleena,Kirby,F,paid
36,67,Colm,Santana,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit="ms"), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.